In [1]:
from pickle_workaround import pickle_load
from TfidfVectorizer import TfidfVectorizer
from vectorizer import tokenize
from nlp import get_top_10_docs

import numpy as np

In [2]:
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [3]:
from data import get_query_corpus, get_connection
_ = get_connection("/home/mark/Documents/missionmark/missionmark_db_creds")

Connecting to Postgres database...
 -> Connection successful!


In [4]:
query = """SELECT opportunity_id, program_description
           FROM import.govwin_opportunity
           WHERE LENGTH(program_description) > 100
        """
doc_ids, corpus = get_query_corpus(query, True)

Loading corpus...
 -> 8847 documents loaded!


In [5]:
tfidf = TfidfVectorizer(pickle_load("../output/CountVectorizer.pkl"), pickle_load("../output/TfidfTransformer.pkl"))
tfidf_corpus = tfidf.transform(corpus)

nmf = pickle_load("../output/NMF.pkl")
H = nmf.components_
W = nmf.transform(tfidf_corpus)
W_max = W.max()

Loading ../output/CountVectorizer.pkl from cache...
 -> ../output/CountVectorizer.pkl loaded!
Loading ../output/TfidfTransformer.pkl from cache...
 -> ../output/TfidfTransformer.pkl loaded!
Loading ../output/NMF.pkl from cache...                                         
 -> ../output/NMF.pkl loaded!


In [6]:
for doc, strength in get_top_10_docs(W, 26):
    print("STRENGTH:", strength)
    print(corpus[doc])
    print("*************************************************************\n")

STRENGTH: 0.24282467640147634
 The Department of Veterans Affairs, Office of Acquisition & Logistics, National Acquisition Center has a requirement for Indefinite Delivery Indefinite Quantity (IDIQ) Multiple Award task order contract (MATOC) for construction to be performed at Building 37 in Hines, IL. REQUIREMENTS  The contractor shall fulfill work included in the task orders may consist of multiple disciplines of construction, and shall include but not be limited to the following categories of work: construction, repair and alteration of facilities, interior and exterior renovations, heating and air-conditioning, HVAC controls, plumbing, fire suppression, interior and exterior electrical and lighting, fire and intrusion alarms, communications, limited utilities, site-work, landscaping, fencing, masonry, roofing, concrete, asphalt paving, painting, storm drainage, limited environmental remediation, concrete and asphalt paving, demolition of facilities, construction of new facilities, 